In [2599]:
import pandas as pd 
import numpy as np 


#instead of trying to create individual features, use def functions and own algo to get the training dataset. 


In [2600]:
Results = pd.read_csv('/Users/anuragshrestha/Desktop/2020-Mens-Data/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
Results = Results.loc[lambda Results: Results['Season']>= 2003,:]

Train = Result[['Season', 'WTeamID', 'LTeamID']]

Seeds = pd.read_csv('/Users/anuragshrestha/Desktop/2020-Mens-Data/MDataFiles_Stage1/MNCAATourneySeeds.csv')

Seeds['Seed'] = pd.to_numeric(Seeds['Seed'].str[1:3], downcast='integer',errors='coerce')



     
  Combining the seeds and results 

In [2601]:
training_set = pd.DataFrame()
training_set['Result'] = np.random.randint(0,2,len(Results.index))
training_set['Season'] = Results['Season'].values
training_set['Team1'] = training_set['Result'].values * Results['WTeamID'].values +(1-training_set['Result'].values) * Results['LTeamID'].values 
training_set['Team2'] = (1-training_set['Result'].values) * Results['WTeamID'].values + training_set['Result'].values * Results['LTeamID'].values

training_set = training_set.merge(Seeds, how='left', left_on=['Season','Team1'], right_on = ['Season','TeamID'])
training_set = training_set.merge(Seeds, how='left', left_on=['Season','Team2'], right_on = ['Season','TeamID'])
training_set['deltaSeed'] = training_set['Seed_x'] - training_set['Seed_y']
training_set.drop(columns = ['Seed_x','TeamID_x','Seed_y','TeamID_y'], inplace=True)

   Teams wins, losses, total games, rankings 
   home wins - .6
   away wins - 1.4

In [2602]:
Stats_detailed = pd.read_csv('/Users/anuragshrestha/Desktop/2020-Mens-Data/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
Stats_compacted = pd.read_csv("/Users/anuragshrestha/Desktop/2020-Mens-Data/MDataFiles_Stage1/MRegularSeasonCompactResults.csv")


#creating amount of wins for Team 1 and Team 2 

Wins = Stats_detailed.groupby(['Season','WTeamID']).size()
Losses = Stats_detailed.groupby(['Season', 'LTeamID']).size()
Records = pd.DataFrame({"Wins": Wins, "Losses": Losses}).reset_index().rename(columns = {'level_0':'Season',
                                                                                         'level_1': 'TeamID'})

Records.isnull().any() #some missing values 

nan_values = Records[Records['Wins'].isnull()]

Records = Records.fillna(0)

Records['Games'] = Records['Wins'] + Records['Losses'] #each team plays a different number of games 


team_id = Records['TeamID']



In [2603]:

#create stats of each teamID per season, since  


def stats_per_season(df, team_id): 
        d = df.copy()
        stats_data = d.loc[(d.WTeamID == team_id) | (d.LTeamID == team_id),:]
        stats_data.sort_values(['Season'], inplace=True)
        stats_data.drop_duplicates(['Season'], keep='last', inplace=True)
    
        WeFG = (stats_data['WFGM']+ stats_data['WFGM3']*.5)/ stats_data['WFGA']
        LeFG = (stats_data['LFGM']+ stats_data['LFGM3']*.5)/ stats_data['LFGA']
        eFG = (WeFG + LeFG)/2  #FG when games won + FG when games lost, average of both for the season 
        
        WPoss = (stats_data['WFGA'] - stats_data['WOR']+ stats_data['WTO'] +.475*stats_data['WFTA'])
        LPoss = (stats_data['LFGA'] - stats_data['LOR']+ stats_data['LTO'] +.475*stats_data['LFTA'])
        Poss = (WPoss + LPoss)/2 #Possesion of the basketball 
        
        WTO_Percent = (stats_data['WTO']/ Poss)  #turnover
        LTO_Percent = (stats_data['LTO']/ Poss)
        TO_Percent = (WTO_Percent+LTO_Percent)/2
        
        WFTrate = (stats_data['WFTA'] / stats_data['WFGA']) 
        LFTrate = (stats_data['LFTA'] / stats_data['LFGA'])
        FTrate = (WFTrate +LFTrate)/2 
        
        WORp = (stats_data['WOR'] / (stats_data['WOR'] +stats_data['WDR']))
        LORp = (stats_data['LOR'] / (stats_data['LOR'] +stats_data['LDR']))
        ORp = (WORp + LORp)/2
        
        
        WORtg = (stats_data['WScore']/ Poss)#points scored/ poss 
        LORtg = (stats_data['LScore']/ Poss)
    
        WDRtg = (stats_data['LScore'] / Poss)
        LDRtg = (stats_data['WScore']/ Poss)
    
        #adjusted offense and defensive ratings based on total games 

        out = pd.DataFrame({'Season': stats_data.Season ,'TeamID': team_id,'eFG': eFG,'Poss': Poss, 'TO%':TO_Percent,
                            'FT%': FTrate,'OR%': ORp,
                            'WORtg': WORtg, 'LORtg': LORtg, 'WDRtg': WDRtg, 'LDRtg': LDRtg})
        return out
    
    

In [2604]:
#df_list = [stats_per_season(Stats_detailed,i) for i in team_id]
#df = pd.concat(df_list)
#df = df.drop_duplicates()

#df.to_csv('df.csv')



In [2605]:
half_stats = pd.read_csv('df.csv', index_col=0)
half_stats= half_stats.reset_index(drop=True)
half_stats

ratings = half_stats.merge(Records[['Season', 'TeamID','Wins','Losses','Games']], how ='left', left_on = ['Season', 'TeamID'], 
                                right_on = ['Season','TeamID'], copy =False )
ratings['AvgORtg'] =(ratings['WORtg']*ratings['Wins'] + ratings['LORtg']*ratings['Losses']) / ratings['Games']
ratings['AvgDRtg'] = (ratings['WDRtg']*ratings['Wins'] + ratings['LDRtg']*ratings['Losses'])/ ratings['Games']
ratings = ratings.drop(columns = ['WORtg','LORtg','WDRtg','LDRtg'])
ratings = ratings[['Season','TeamID', 'Wins','Losses','Games','eFG','Poss','TO%','FT%','OR%','AvgORtg','AvgDRtg']]

#Rankings of pomeroy 

Rank = pd.read_csv("/Users/anuragshrestha/Desktop/2020-Mens-Data/MDataFiles_Stage1/MMasseyOrdinals.csv")
RankPOM = Rank[Rank['SystemName'] == 'POM']
RankPOM = RankPOM.groupby(['TeamID', 'Season']).mean().reset_index()
ratings = ratings.merge(RankPOM[['Season','TeamID','OrdinalRank']], how ='left',left_on=['TeamID','Season'], 
                        right_on = ['TeamID','Season'], copy = False)

ratings = ratings.drop_duplicates().reset_index(drop=True)
ratings

,Season,TeamID,Wins,Losses,Games,eFG,Poss,TO%,FT%,OR%,AvgORtg,AvgDRtg,OrdinalRank
0,2003,1102,12.0,16.0,28.0,0.460027,58.7625,0.255265,0.468022,0.283482,0.872763,0.914091,140.266667
1,2004,1102,22.0,6.0,28.0,0.571899,57.9000,0.241796,0.638254,0.225953,1.093018,1.014064,79.166667
2,2005,1102,17.0,12.0,29.0,0.564583,56.6500,0.220653,0.318750,0.256979,1.111483,1.059744,53.888889
3,2006,1102,22.0,6.0,28.0,0.583158,57.4625,0.269741,0.261579,0.308528,1.177165,0.928556,45.000000
4,2007,1102,22.0,8.0,30.0,0.619634,58.8500,0.178420,0.598834,0.266176,1.301614,1.111300,22.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5829,2017,1262,10.0,20.0,30.0,0.412381,75.1125,0.246297,0.452222,0.321212,0.838742,0.878682,281.055556
5830,2018,1262,11.0,18.0,29.0,0.540903,72.0250,0.242971,0.338798,0.317151,0.970927,1.014494,290.833333
5831,2019,1262,13.0,17.0,30.0,0.568480,62.2875,0.160546,0.337938,0.341176,1.179477,1.196602,258.947368
5832,2019,1465,12.0,14.0,26.0,0.476136,64.8875,0.084762,0.231061,0.279872,1.056266,1.070492,211.947368


In [2606]:
##training set 


cols = ['eFG','Poss','TO%', 'FT%', 'OR%','AvgORtg', 'AvgDRtg', 'OrdinalRank']

training_set = training_set.merge(ratings[['Season','TeamID']+cols], how='left', left_on=['Season','Team1'], right_on = ['Season','TeamID'])
training_set = training_set.merge(ratings[['Season','TeamID']+cols], how='left', left_on=['Season','Team2'], right_on = ['Season','TeamID'],suffixes = ['_1','_2'])
training_set.drop(columns=['TeamID_1','TeamID_2'], inplace=True)

training_set
                                                                           

,Result,Season,Team1,Team2,deltaSeed,eFG_1,Poss_1,TO%_1,FT%_1,OR%_1,...,AvgDRtg_1,OrdinalRank_1,eFG_2,Poss_2,TO%_2,FT%_2,OR%_2,AvgORtg_2,AvgDRtg_2,OrdinalRank_2
0,0,2003,1411,1421,0,0.563612,73.0875,0.198392,0.451887,0.465998,...,1.195827,260.133333,0.531791,77.4250,0.200194,0.376894,0.285362,1.035039,1.083139,259.400000
1,1,2003,1112,1436,-15,0.530769,73.8375,0.148976,0.253846,0.260504,...,0.981886,3.133333,0.473604,77.6500,0.264005,0.233398,0.252778,0.914803,0.810889,164.533333
2,0,2003,1272,1113,-3,0.479621,74.8500,0.146961,0.425612,0.239060,...,0.996936,46.333333,0.503160,61.8250,0.266882,0.608190,0.402198,1.027929,0.961557,43.933333
3,1,2003,1141,1166,5,0.349615,65.9875,0.265202,0.572654,0.489368,...,0.816769,96.066667,0.511856,69.1625,0.166275,0.400615,0.307143,1.157134,1.069506,14.866667
4,0,2003,1301,1143,1,0.491671,78.9000,0.221800,0.390137,0.329770,...,0.975919,50.866667,0.586735,68.1750,0.256692,0.248190,0.398467,1.120848,1.035368,42.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,0,2019,1246,1120,-3,0.566964,64.4500,0.131885,0.392857,0.344289,...,1.221525,11.263158,0.526425,69.0750,0.173724,0.291513,0.229167,1.139426,1.003172,12.368421
1111,0,2019,1181,1277,-1,0.445022,69.2875,0.180408,0.317893,0.310023,...,0.930479,1.631579,0.486007,60.1500,0.124688,0.263102,0.196461,1.065963,1.012175,5.789474
1112,0,2019,1277,1403,-1,0.486007,60.1500,0.124688,0.263102,0.196461,...,1.012175,5.789474,0.474535,74.5000,0.214765,0.322074,0.332576,1.047819,1.005872,11.842105
1113,1,2019,1438,1120,-4,0.546349,58.6500,0.179028,0.285685,0.303571,...,1.021952,2.052632,0.526425,69.0750,0.173724,0.291513,0.229167,1.139426,1.003172,12.368421


In [2607]:
for col in cols:
    name = 'delta'+col
    training_set[name] = training_set[col+'_1'] - training_set[col+'_2']
    training_set.drop(columns=[col+'_1',col+'_2'],inplace=True)



training_set.to_csv('training_set.csv')
training_set


,Result,Season,Team1,Team2,deltaSeed,deltaeFG,deltaPoss,deltaTO%,deltaFT%,deltaOR%,deltaAvgORtg,deltaAvgDRtg,deltaOrdinalRank
0,0,2003,1411,1421,0,0.031821,-4.3375,-0.001801,0.074993,0.180636,0.190888,0.112688,0.733333
1,1,2003,1112,1436,-15,0.057165,-3.8125,-0.115029,0.020448,0.007726,0.216058,0.170997,-161.400000
2,0,2003,1272,1113,-3,-0.023539,13.0250,-0.119922,-0.182578,-0.163137,-0.007498,0.035379,2.400000
3,1,2003,1141,1166,5,-0.162241,-3.1750,0.098927,0.172039,0.182225,-0.215995,-0.252737,81.200000
4,0,2003,1301,1143,1,-0.095064,10.7250,-0.034893,0.141948,-0.068698,-0.106906,-0.059449,8.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,0,2019,1246,1120,-3,0.040540,-4.6250,-0.041839,0.101344,0.115122,0.121593,0.218353,-1.105263
1111,0,2019,1181,1277,-1,-0.040985,9.1375,0.055719,0.054792,0.113562,-0.033606,-0.081696,-4.157895
1112,0,2019,1277,1403,-1,0.011472,-14.3500,-0.090077,-0.058972,-0.136115,0.018144,0.006303,-6.052632
1113,1,2019,1438,1120,-4,0.019925,-10.4250,0.005304,-0.005828,0.074405,0.021059,0.018780,-10.315789
